In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

In [34]:
qubo = QuadraticProgram()

n = 2 # liczba lekarzy
m = 3 # liczba pacjentów

qubo.binary_var("x11")
qubo.binary_var("x12")
qubo.binary_var("x13")
qubo.binary_var("x21")
qubo.binary_var("x22")
qubo.binary_var("x23")

print(qubo)

minimize 0 (6 variables, 0 constraints, '')


In [32]:
qubo.minimize(linear=[1, -2, 3], quadratic={("x", "y"): 1, ("x", "z"): -1, ("y", "z"): 2})
print(qubo.prettyprint())

Problem name: my problem

Minimize
  2*x*y + x*z + y*z - z^2 + x + 3

Subject to
  Linear constraints (3)
    x + 2*y == 3  'lin_eq'
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Quadratic constraints (3)
    x^2 - y*z + x + y == 1  'quad_eq'
    x^2 - y*z + x + y <= 1  'quad_leq'
    x^2 - y*z + x + y >= 1  'quad_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x


In [40]:
# Make a Docplex model
from docplex.mp.model import Model

w1 = 30
w2 = 40
w3 = 60

mdl = Model("docplex model")
x11 = mdl.binary_var("x11")
x12 = mdl.binary_var("x12")
x13 = mdl.binary_var("x13")
x21 = mdl.binary_var("x21")
x22 = mdl.binary_var("x22")
x23 = mdl.binary_var("x23")
mdl.minimize(((x11*w1+x12*w2+x13*w3)-(x21*w1+x22*w2+x23*w3))*((x11*w1+x12*w2+x13*w3)-(x21*w1+x22*w2+x23*w3))-(x11-x21)*(x11-x21)*130-(x12-x22)*(x12-x22)*130-(x13-x23)*(x13-x23)*130)
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: [ 1540 x11^2 + 4800 x11*x12 + 7200 x11*x13 - 3080 x11*x21 - 4800 x11*x22
      - 7200 x11*x23 + 2940 x12^2 + 9600 x12*x13 - 4800 x12*x21 - 5880 x12*x22
      - 9600 x12*x23 + 6940 x13^2 - 7200 x13*x21 - 9600 x13*x22 - 13880 x13*x23
      + 1540 x21^2 + 4800 x21*x22 + 7200 x21*x23 + 2940 x22^2 + 9600 x22*x23
      + 6940 x23^2 ]/2
Subject To

Bounds
 0 <= x11 <= 1
 0 <= x12 <= 1
 0 <= x13 <= 1
 0 <= x21 <= 1
 0 <= x22 <= 1
 0 <= x23 <= 1

Binaries
 x11 x12 x13 x21 x22 x23
End


In [41]:
mod = from_docplex_mp(mdl)
print(type(mod))
print()
print(mod.prettyprint())

<class 'qiskit_optimization.problems.quadratic_program.QuadraticProgram'>

Problem name: docplex model

Minimize
  770*x11^2 + 2400*x11*x12 + 3600*x11*x13 - 1540*x11*x21 - 2400*x11*x22
  - 3600*x11*x23 + 1470*x12^2 + 4800*x12*x13 - 2400*x12*x21 - 2940*x12*x22
  - 4800*x12*x23 + 3470*x13^2 - 3600*x13*x21 - 4800*x13*x22 - 6940*x13*x23
  + 770*x21^2 + 2400*x21*x22 + 3600*x21*x23 + 1470*x22^2 + 4800*x22*x23
  + 3470*x23^2

Subject to
  No constraints

  Binary variables (6)
    x11 x12 x13 x21 x22 x23


In [42]:
op, offset = mod.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 2855.0
operator:
SparsePauliOp(['IIIIZZ', 'IIIZIZ', 'IIZIIZ', 'IZIIIZ', 'ZIIIIZ', 'IIIZZI', 'IIZIZI', 'IZIIZI', 'ZIIIZI', 'IIZZII', 'IZIZII', 'ZIIZII', 'IZZIII', 'ZIZIII', 'ZZIIII'],
              coeffs=[  600.+0.j,   900.+0.j,  -385.+0.j,  -600.+0.j,  -900.+0.j,  1200.+0.j,
  -600.+0.j,  -735.+0.j, -1200.+0.j,  -900.+0.j, -1200.+0.j, -1735.+0.j,
   600.+0.j,   900.+0.j,  1200.+0.j])


In [43]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.prettyprint())

Problem name: 

Minimize
  2400*x0*x1 + 3600*x0*x2 - 1540*x0*x3 - 2400*x0*x4 - 3600*x0*x5 + 4800*x1*x2
  - 2400*x1*x3 - 2940*x1*x4 - 4800*x1*x5 - 3600*x2*x3 - 4800*x2*x4 - 6940*x2*x5
  + 2400*x3*x4 + 3600*x3*x5 + 4800*x4*x5 + 770*x0 + 1470*x1 + 3470*x2 + 770*x3
  + 1470*x4 + 3470*x5

Subject to
  No constraints

  Binary variables (6)
    x0 x1 x2 x3 x4 x5


In [45]:
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np

In [46]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

In [47]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [49]:
exact_result = exact.solve(qp)
print(exact_result.prettyprint())

objective function value: -290.0
variable values: x0=1.0, x1=1.0, x2=0.0, x3=0.0, x4=0.0, x5=1.0
status: SUCCESS
